In [49]:
## Load essential packages ##

%reset -f

import os
import openai
import pandas as pd

In [50]:
# Setup the Openai client to query the embeddings API 


from openai import OpenAI

client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"), organization = os.getenv("OPENAI_ORGANIZATION"))

def request_completionT(prompt):

    response = client.embeddings.create(
        input=prompt,
        model="text-embedding-3-large"
    )
    
    return response.data[0].embedding


In [51]:
# Open the datafile, read the responses, and put into a format to pass to the OpenAI client for embeddings

import csv
from io import StringIO

#RealResponses = pd.read_csv('reasons_nonvx_otherAlt.csv',encoding='utf-8') 
RealResponses = pd.read_csv('ecv_reasons_novx 1Clean.csv',encoding='ISO-8859-1') 

with open('ecv_reasons_novx 1.csv', 'r', encoding='utf-8', errors='ignore') as file:
    content = file.read()

# Assuming 'content' is your CSV file content as a string
content_io = StringIO(content)

# Create a CSV DictReader object
reader = csv.DictReader(content_io)

rows = list(reader)

#print(RealResponses.head(10))

In [ ]:
# Acquire the embeddings for each response in the dataset.  Note that you can pass multiple responses at once, but here I've only passed one at a time.

import numpy as np
import re

rowsData = []

for index in range(0,len(RealResponses)):

    batchStrings = str(rows[index]['reason'])
    assistantResponse = request_completionT(batchStrings)

    rowsData.append(assistantResponse)
    print(index)

matrix = np.array(rowsData)

# Save the array to a CSV file
np.savetxt("embeddingsLarge_LargeResponseSet.csv", matrix, delimiter=',', fmt='%.15f')
